<a href="https://colab.research.google.com/github/zenosance/machine-learning/blob/main/week9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 1
Klasisfikasi Iris dengan Perceptron

## Deskripsi
Pada praktikum ini, Anda diminta untuk melakukan klasifikasi data iris dengan menggunakan model perceptron. Jadi kita akan menggunakan dataset iris pada praktikum sebelumnya.

Untuk menambah pemahaman proses akan dilakukan secara manual.

## Langkah 1: Import Library


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Langkah 2: Load Data dan Visualisasi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/machine-learning/data/iris.csv', header=None)

Mounted at /content/drive


NameError: ignored

In [ ]:
setosa = df[df[4] == 'Iris-setosa']
versicolor = df[df[4] == 'Iris-versicolor']
virginica = df[df[4] == 'Iris-virginica']

a, b = 0, 3
plt.scatter(setosa[a], setosa[b], color='red', marker='o', label='setosa')
plt.scatter(versicolor[a], versicolor[b], color='blue', marker='x', label='versicolor')

plt.xlabel('Petal Length')
plt.ylabel('Sepal Length')
plt.legend(loc='upper left')
plt.grid()
plt.show()

## Langkah 3: Membuat Kelas Perceptron

## Langkah 4: Pilih Data dan Encoding Label

## Langkah 5: Fitting Model

## Langkah 6: Visualisasi Nilai Error Per Epoch

## Langkah 7: Visualisasi Decision Boundary